In [1]:
import json,requests
marquez_url = "http://api:5000" ## this may depend on your local setup
if (requests.get("{}/api/v1/namespaces".format(marquez_url)).status_code == 200):
    print("Marquez is OK.")
else:
    print("Cannot connect to Marquez")

Marquez is OK.


Marquez URL is http://localhost:3000

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.master('local')
         .appName('Jelly Data Mesh')
         .config('spark.extraListeners', 'io.openlineage.spark.agent.OpenLineageSparkListener')
         .config('spark.jars.packages', 'io.openlineage:openlineage-spark:0.20.6')
         .config('spark.openlineage.transport.type', 'http')
         .config('spark.openlineage.transport.url', 'http://api:5000')    
         .config('spark.openlineage.namespace', 'jelly-world')
         .getOrCreate())

spark.sparkContext.setLogLevel('DEBUG')

## Initialise some datasets

In [3]:
spark.sql('DROP TABLE IF EXISTS jelly_colors')
spark.sql('DROP TABLE IF EXISTS jelly_transactions')
spark.sql('DROP TABLE IF EXISTS jelly_users')

DataFrame[]

In [4]:
!rm -rf /home/jovyan/notebooks/spark-warehouse/jelly_*

In [5]:
import uuid
greenUuid = str(uuid.uuid4())
spark.createDataFrame([
    {'color_uuid': greenUuid, 'name': 'green'},
    {'color_uuid': str(uuid.uuid4()), 'name': 'yellow'},
]).write.mode("overwrite").saveAsTable('jelly_colors')

pawelUuid = str(uuid.uuid4())
maciejUuid = str(uuid.uuid4());

spark.createDataFrame([
    {'uuid': pawelUuid, 'name': 'pawel@jellyworld.pl'},
    {'uuid': maciejUuid, 'name': 'maciej@jellyworld.pl'},
]).write.mode("overwrite").saveAsTable('jelly_users')

spark.createDataFrame([
    {
        'from_user_uuid': pawelUuid, 
        'to_user_uuid': maciejUuid, 
        'exchanched_color_uuid': greenUuid, 
        'amount_exchanged': 5, 'lat': '52.2278015' , 
        'lng':'21.0022384', 
        'exchange_date': '2023-02-29 12:00:00.000' 
    }
]).write.mode("overwrite").saveAsTable('jelly_transactions')

## Active daily users report table

In [6]:
spark.sql('''
  SELECT u.name, SUM(amount_exchanged)
  FROM jelly_users u
  JOIN jelly_transactions t ON (u.uuid=t.from_user_uuid OR u.uuid=t.to_user_uuid) 
  GROUP BY u.name
''').write.mode("overwrite").saveAsTable('jelly_daily_active_users')

http://localhost:3000/lineage/dataset/file/%2Fhome%2Fjovyan%2Fnotebooks%2Fspark-warehouse%2Fjelly_daily_active_users